In [32]:
import pandas as pd
import numpy as np
data=pd.read_csv("bike_marketing.csv")

In [33]:
data.head(3)

,company_num,google_adwords,facebook,twitter,marketing_total,revenues,employees,pop_density
0,1,65.66,47.86,52.46,166,39.26,5,High
1,2,39.10,55.20,77.40,172,38.90,7,Medium
2,3,174.81,52.01,68.01,295,49.51,11,Medium


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   company_num      172 non-null    int64  
 1   google_adwords   172 non-null    float64
 2   facebook         172 non-null    float64
 3   twitter          172 non-null    float64
 4   marketing_total  172 non-null    int64  
 5   revenues         172 non-null    float64
 6   employees        172 non-null    int64  
 7   pop_density      172 non-null    object 
dtypes: float64(4), int64(3), object(1)
memory usage: 10.9+ KB


In [35]:
data['pop_density']=data['pop_density'].astype('category')

In [36]:
low=data.loc[data['pop_density']=='Low', 'revenues']
medium=data.loc[data['pop_density']=='Medium', 'revenues']
high=data.loc[data['pop_density']=='High', 'revenues']

H0 : pop_density 별 매출에 차이가 없다.
    
H1 : pop_density 집단 간 매출이 모두 같다고 할수는 없다.

In [37]:
import scipy.stats as stats
stats.shapiro(low)

ShapiroResult(statistic=0.9840328693389893, pvalue=0.5353288650512695)

In [38]:
stats.shapiro(medium)

ShapiroResult(statistic=0.9450081586837769, pvalue=0.017985567450523376)

In [39]:
stats.shapiro(high)

ShapiroResult(statistic=0.9552056789398193, pvalue=0.04857487604022026)

In [40]:
stats.kruskal(low, medium, high)

KruskalResult(statistic=0.900557928910252, pvalue=0.6374503008400596)

귀무가설을 기각할 수 없다. 즉 pop_density 별 매출에 차이가 없다.

2. 회귀분석(전진선택법)

In [41]:
data

,company_num,google_adwords,facebook,twitter,marketing_total,revenues,employees,pop_density
0,1,65.66,47.86,52.46,166,39.26,5,High
1,2,39.10,55.20,77.40,172,38.90,7,Medium
2,3,174.81,52.01,68.01,295,49.51,11,Medium
3,4,34.36,61.96,86.86,183,40.56,7,High
4,5,78.21,40.91,30.41,150,40.21,9,Low
...,...,...,...,...,...,...,...,...
167,168,188.45,51.05,11.45,251,48.95,9,Low
168,169,174.07,47.37,16.57,238,49.37,12,High
169,170,59.86,12.76,21.66,94,36.96,4,Low
170,171,117.91,16.01,18.01,152,41.11,7,Low


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   company_num      172 non-null    int64   
 1   google_adwords   172 non-null    float64 
 2   facebook         172 non-null    float64 
 3   twitter          172 non-null    float64 
 4   marketing_total  172 non-null    int64   
 5   revenues         172 non-null    float64 
 6   employees        172 non-null    int64   
 7   pop_density      172 non-null    category
dtypes: category(1), float64(4), int64(3)
memory usage: 9.8 KB


In [43]:
pd.get_dummies(data)

,company_num,google_adwords,facebook,twitter,marketing_total,revenues,employees,pop_density_High,pop_density_Low,pop_density_Medium
0,1,65.66,47.86,52.46,166,39.26,5,1,0,0
1,2,39.10,55.20,77.40,172,38.90,7,0,0,1
2,3,174.81,52.01,68.01,295,49.51,11,0,0,1
3,4,34.36,61.96,86.86,183,40.56,7,1,0,0
4,5,78.21,40.91,30.41,150,40.21,9,0,1,0
...,...,...,...,...,...,...,...,...,...,...
167,168,188.45,51.05,11.45,251,48.95,9,0,1,0
168,169,174.07,47.37,16.57,238,49.37,12,1,0,0
169,170,59.86,12.76,21.66,94,36.96,4,0,1,0
170,171,117.91,16.01,18.01,152,41.11,7,0,1,0


In [44]:
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

value="+".join(list(data.drop(columns=['company_num', 'revenues'])))

In [45]:
model=ols(f'revenues~{value}', data=data)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               revenues   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                     163.1
Date:                Tue, 20 Sep 2022   Prob (F-statistic):           1.80e-70
Time:                        00:41:31   Log-Likelihood:                -368.53
No. Observations:                 172   AIC:                             753.1
Df Residuals:                     164   BIC:                             778.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                28.6894      0.659     43.520      0.000      27.388      29.991
pop_density[T.Low]       -0.2822      0.397     -0.710      0.479      -1.067       0.502
pop_density[T.Medium]    -0.4908      0.416     -1.180      0.240      -1.312       0.330
google_adwords            1.1783      0.578      2.038      0.043       0.037       2.320
facebook                  1.3132      0.579      2.269      0.025       0.170       2.456
twitter                   1.1396      0.579      1.969      0.051      -0.003       2.282
marketing_total          -1.1366      0.578     -1.966      0.051      -2.278       0.005
employees                 0.3834      0.105      3.653      0.000       0.176       0.591
==============================================================================
Omnibus:                        4.019   Durbin-Watson:                   2.091
Prob(Omnibus):                  0.134   Jarque-Bera (JB):                3.986
Skew:                          -0.333   Prob(JB):                        0.136
Kurtosis:                       2.663   Cond. No.                     2.35e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.35e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [46]:
from patsy import dmatrices
y, X=dmatrices(f'revenues~{value}', data=data, return_type='dataframe')

In [51]:
import time 
import itertools
import statsmodels.api as sm
import statsmodels.formula.api as smf

def processSubset(X,y, feature_set):
    model = sm.OLS(y, X[list(feature_set)])
    regr = model.fit()
    AIC = regr.aic
    return {'model':regr, 'AIC':AIC}

#전진선택법
def forward(X,y,predictors):
    remaining_predictors = [p for p in X.columns.difference(['Intercept']) if p not in predictors]
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(X=X, y=y, feature_set=predictors + [p] + ['Intercept']))
        
    models = pd.DataFrame(results)
    
    best_model = models.loc[models['AIC'].argmin()]
    print('Proceed', models.shape[0], 'models on', len(predictors)+1, 'predictors in')
    print('Selected predictors:', best_model['model'].model.exog_names, 'AIC:', best_model[0])
    return best_model

#후진소거법
def backward(X,y,predictors):
    tic = time.time()
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors) -1):
        results.append(processSubset(X=X, y=y, feature_set=list(combo) + ['Intercept']))
    
    models = pd.DataFrame(results)
    best_model= models.loc[models['AIC'].argmin()]
    toc = time.time()
    
    print('Processed', models.shape[0], 'models on', len(predictors)-1, 'predictors in', (toc-tic))
    print('Selected predictors:', best_model['model'].model.exog_names,
          'AIC:', best_model[0])
    return best_model

#단계적 선택법
def Stepwise_model(X,y):
    Stepmodels = pd.DataFrame(columns=['AIC','model'])
    tic = time.time()
    predictors = []
    Smodel_before = processSubset(X,y, predictors+['Intercept'])['AIC']
    
    for i in range(1, len(X.columns.difference(['Intercept']))+1):
        Forward_result = forward(X=X, y=y, predictors=predictors)
        print('forward')
        Stepmodels.loc[i] = Forward_result
        predictors = Stepmodels.loc[i]['model'].model.exog_names
        predictors = [k for k in predictors if k!= 'Intercept']
        Backward_result = backward(X=X, y=y, predictors=predictors)
        
        if Backward_result['AIC'] < Forward_result['AIC']:
            Stepmodels.loc[i] = Backward_result
            predictors = Stepmodels.loc[i]['model'].model.exog_names
            Smodel_before = Stepmodels.loc[i]['AIC']
            predictors = [k for k in predictors if k!='Intercept']
            print('backward')
            
        if Stepmodels.loc[i]['AIC'] > Smodel_before:
            break
        
        else:
            Smodel_before = Stepmodels.loc[i]['AIC']
            
    toc = time.time()
    print('Total elapsed time : ',(toc-tic), 'seconds') 
    
    return (Stepmodels['model'][len(Stepmodels['model'])])

In [52]:
predictors=[]
forward_best_model=forward(X, y, predictors)

Proceed 7 models on 1 predictors in
Selected predictors: ['marketing_total', 'Intercept'] AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000001BEB9C33760>


In [54]:
forward_best_model.summary()

AttributeError: 'Series' object has no attribute 'summary'

In [56]:
best_model[0]

NameError: name 'best_model' is not defined